In [ ]:
!sudo apt-get update
!sudo apt-get install tesseract-ocr

In [ ]:
pip install pytesseract

In [ ]:
pip install pillow

In [ ]:
pip install dateparser

In [ ]:
pip install deskew

In [157]:
import pytesseract
import math
from typing import Tuple, Union
from deskew import determine_skew
import cv2
import dateparser
from dateparser.search import search_dates
import matplotlib.pyplot as plt
import numpy as np
import os

In [220]:
name = "25AUG2015"

In [178]:
os.mkdir(name)

In [221]:
dir = name + ".jpg"

# Rotates the image so texts are aligned


In [222]:
def rotate(
        image: np.ndarray, angle: float, background: Union[int, Tuple[int, int, int]]
) -> np.ndarray:
    old_width, old_height = image.shape[:2]
    angle_radian = math.radians(angle)
    width = abs(np.sin(angle_radian) * old_height) + abs(np.cos(angle_radian) * old_width)
    height = abs(np.sin(angle_radian) * old_width) + abs(np.cos(angle_radian) * old_height)

    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    rot_mat[1, 2] += (width - old_width) / 2
    rot_mat[0, 2] += (height - old_height) / 2
    return cv2.warpAffine(image, rot_mat, (int(round(height)), int(round(width))), borderValue=background)

image = cv2.imread(dir)
grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
angle = determine_skew(grayscale)
rotated = rotate(image, angle, (0, 0, 0))
cv2.imwrite(name + '/' + 'deskew.png', rotated)
img = rotated

# Binarize Image

In [223]:
im_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
th, im_Binarize = cv2.threshold(im_gray, 128, 192, cv2.THRESH_OTSU)
cv2.imwrite(name + '/' + 'binarized.png', im_Binarize)
img = im_Binarize

# Morphological Transformations

Erossion

In [224]:
kernel = np.ones((5,5),np.uint8)

In [225]:
erosion = cv2.erode(dilation,kernel,iterations = 1)
cv2.imwrite(name + '/' + 'erosion.png', erosion)

True

Dilation

In [226]:
dilation = cv2.dilate(img,kernel,iterations = 1)
cv2.imwrite(name + '/' + 'dilation.png', dilation)

True

Opening/ Errosion-Dilation

In [227]:
opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
cv2.imwrite(name + '/' + 'opening.png', opening)

True

Closing/ Dilation-Erosion

In [228]:
closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
cv2.imwrite(name + '/' + 'closing.png', closing)

True

Selected transformation

In [231]:
img = opening

# Date Extraction

In [232]:
cv2.imwrite(name + '/' + 'final.png', img)

True

In [ ]:
string = pytesseract.image_to_string(img)
print(string)

In [208]:
search_dates(string)

[('15APR2016', datetime.datetime(2016, 4, 15, 0, 0))]

In [195]:
# Desired Output
search_dates('Expire: 15APR2016 bla bla bla')

[('15APR2016', datetime.datetime(2016, 4, 15, 0, 0))]

In [234]:
!zip -r /content/25AUG2015.zip /content/25AUG2015/

  adding: content/25AUG2015/ (stored 0%)
  adding: content/25AUG2015/deskew.png (deflated 2%)
  adding: content/25AUG2015/erosion.png (deflated 24%)
  adding: content/25AUG2015/dilation.png (deflated 23%)
  adding: content/25AUG2015/opening.png (deflated 23%)
  adding: content/25AUG2015/binarized.png (deflated 18%)
  adding: content/25AUG2015/final.png (deflated 23%)
  adding: content/25AUG2015/closing.png (deflated 26%)


# Pre-Modified Extraction

In [199]:
name = '25AUG2015'

In [200]:
preImg = cv2.imread(name + '/' + 'final.png')

In [ ]:
string = pytesseract.image_to_string(preImg)
print(string)
li = search_dates(string)